# Agents for Amazon Bedrock - create agent

This notebook provides sample code for building an Agent for Amazon Bedrock that has an Action Group attached to it.

### Use Case
We will create a assistant that can help you as any chatbot, but with a plus, you will be able to get weather for a City ! 

### Notebook Walk-through

In this notebook we will:
- Create a lambda function that get weather
- Create an agent
- Create an action group and associate it with the agent
- Test the agent invocation


### Pre-requisites
This notebook requires permissions to:
- create and delete Amazon IAM roles
- create lambda functions
- access Amazon Bedrock

If running on SageMaker Studio, you should add the following managed policies to your role:
- IAMFullAccess
- AWSLambda_FullAccess
- AmazonBedrockFullAccess


## Setup
Before running the rest of this notebook, you'll need to run the cells below to ensure necessary libraries loaded

Let's now import the necessary libraries and initiate the required boto3 clients

In [ ]:
import time
import boto3
import logging
import uuid

from utils.agent import create_agent_role, create_lambda_role
from utils.agent import create_lambda, invoke_agent_helper

In [ ]:
session = boto3.session.Session()

In [ ]:
#Clients
s3_client = session.client('s3', region_name=session.region_name)
sts_client = session.client('sts', region_name=session.region_name)
region = session.region_name


In [ ]:
account_id = sts_client.get_caller_identity()["Account"]
bedrock_agent_client = session.client('bedrock-agent', region_name=region)
bedrock_agent_runtime_client = session.client('bedrock-agent-runtime', region_name=region)
logging.basicConfig(format='[%(asctime)s] p%(process)s {%(filename)s:%(lineno)d} %(levelname)s - %(message)s', level=logging.INFO)
logger = logging.getLogger(__name__)
region, account_id

### Setting up Agent's information

We will now set the variables that define our agent:

- **agent_name**: provides the name of the agent to be created
- **agent_description**: the description of the agent used to display the agents list on the console. This description is **not** part of the agent's prompts
- **agent_instruction**: the instructions of what the agent **should** and **should not** do. This description is part of the agent's prompt and is used during the agent's invocation
- **agent_action_group_name**: the action group name used on the definition of the agent's action.
- **agent_action_group_description:**: the description of the action group name used on the UI to list the action groups. This description is **not** used by the agent's prompts

In [ ]:
suffix = f"{region}-{account_id}"
agent_name = "weather-agent"
agent_bedrock_allow_policy_name = f"{agent_name}"
agent_role_name = f"AmazonBedrockExecutionRoleForAgents_{agent_name}"

agent_description = "This agent provides weather information for a given city"
agent_instruction = """
The agent should be capable of providing accurate weather information, including current conditions and forecasts, when prompted. 
The agent must interpret weather data to give practical advice and answer weather-related queries. 
However, it's crucial that the agent maintains its ability to engage in a wide range of topics beyond weather. 
It should seamlessly switch between weather information and other subjects, using context to determine the nature of each query. 
Above all, the agent should remember that while it's capable of providing weather information, this is just one of its many functions, and it should always be prepared to engage with any topic or task presented by the user.
"""

agent_action_group_description = """
This agent provides weather information for a given city. 
"""

agent_action_group_name = "weather-action-group"

### Select Foundation Model
You can find more information about the supported foundation models [here](https://docs.aws.amazon.com/bedrock/latest/userguide/agents-supported.html)

In [ ]:
agent_foundation_model = "anthropic.claude-3-haiku-20240307-v1:0"

### Creating Lambda Function

Next we will create the [AWS Lambda](https://aws.amazon.com/lambda/) function that executes the actions for our agent. This lambda function will have 1 action:
* ```get_weather(city)```: returns the weather for the city


The `lambda_handler` receives the `event` from the agent and the `event` contains information about the `function` to be executed and its `parameters`. 

A `functionResponse` is returned by the lambda function with the response body having a `TEXT` field.

You can find more information on how to set your agent lambda function [here](https://docs.aws.amazon.com/bedrock/latest/userguide/agents-lambda.html).

Let's first write the code of the lambda function to the `lambda_function.py` file

In [ ]:
lambda_function_name = f'{agent_name}-lambda'

In [ ]:
%%writefile annex/agent/lambda_function.py
import json
import uuid
import boto3
import urllib.request
from urllib.parse import quote


def lambda_handler(event, _):
    """
    This function gets the weather information for a given city
    """

    # Extract info from the event
    actionGroup = event.get('actionGroup', '')
    function = event.get('function', '')
    city = event['parameters'][0]['value']
    encoded_city = quote(city)

    # Get the location data based on the city
    url = f'https://geocoding-api.open-meteo.com/v1/search?name={encoded_city}&count=1&language=en&format=json'
    with urllib.request.urlopen(url) as response:
        location_data = json.loads(response.read().decode())
        if not location_data['results']:
            return {"error": "City not found"}
        
        lat = location_data['results'][0]['latitude']
        lon = location_data['results'][0]['longitude']

    # Get the weather data based on the location
    weather_url = f'https://api.open-meteo.com/v1/forecast?latitude={lat}&longitude={lon}&current=temperature_2m,relative_humidity_2m,weather_code,wind_speed_10m&daily=weather_code,temperature_2m_max,temperature_2m_min&timezone=auto'
    with urllib.request.urlopen(weather_url) as response:
        weather_data = json.loads(response.read().decode())

    current = weather_data['current']
    daily = weather_data['daily']

    # Prepare the response
    weather_codes = {
        0: "Clear sky", 1: "Mainly clear", 2: "Partly cloudy", 3: "Overcast",
        45: "Fog", 48: "Depositing rime fog",
        51: "Light drizzle", 53: "Moderate drizzle", 55: "Dense drizzle",
        61: "Slight rain", 63: "Moderate rain", 65: "Heavy rain",
        71: "Slight snow fall", 73: "Moderate snow fall", 75: "Heavy snow fall",
        77: "Snow grains", 80: "Slight rain showers", 81: "Moderate rain showers",
        82: "Violent rain showers", 85: "Slight snow showers", 86: "Heavy snow showers",
        95: "Thunderstorm", 96: "Thunderstorm with slight hail", 99: "Thunderstorm with heavy hail"
    }
    response_core =  {   
        'temperature': current['temperature_2m'],
        'condition': weather_codes.get(current['weather_code'], "Unknown"),
        'humidity': current['relative_humidity_2m'],
        'wind_speed': current['wind_speed_10m'],
        'forecast_max': daily['temperature_2m_max'][0],
        'forecast_min': daily['temperature_2m_min'][0],
        'forecast_condition': weather_codes.get(daily['weather_code'][0], "Unknown")
    }

    responseBody = {'TEXT': {'body': json.dumps(response_core)}}
    action_response = {
        'actionGroup': actionGroup,
        'function': function,
        'functionResponse': {
            'responseBody': responseBody
        }
    }
    function_response = {'response': action_response, 'messageVersion': event['messageVersion']}

    return function_response

    

We Create the log group to store the function logs

In [ ]:
logs_client = session.client("logs")
log_group_name = f"/aws/lambda/{lambda_function_name}"

# Create the log group
# Check if the log group already exists
response = logs_client.describe_log_groups(logGroupNamePrefix=log_group_name)
if any(group['logGroupName'] == log_group_name for group in response['logGroups']):
    logger.info(f"Log group '{log_group_name}' already exists.")
else:
    # If log group does not exist, create it
    logs_client.create_log_group(logGroupName=log_group_name)
    logger.info(f"Log group '{log_group_name}' created successfully.")

Next we create the function requirements for IAM role and policies using the support function `create_lambda_role` and create the lambda using the support function `create_lambda` both from the `agent.py` file

In [ ]:
lambda_iam_role = create_lambda_role(agent_name)

In [ ]:
lambda_function = create_lambda(lambda_function_name, lambda_iam_role)

### Creating Agent

Now that we have created the lambda function, let's create our Agent. 

To do so, we first need to create an agent role and its required policies:
* Invoke model

Let's do so using the `create_agent_role` function from the `agent.py` file.

In [ ]:
agent_role = create_agent_role(agent_name, agent_foundation_model)

With the Agent IAM role created, we can now use the boto3 function [`create_agent`](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/bedrock-agent/client/create_agent.html) to create our agent. 

On the agent creation, all you need to provide is the agent name, foundation model and instruction. We will associate an action group to the agent once it has been created

We will retrieve the `agentId`. It will be used to associate the action group to the agent in our next step.

In [ ]:
response = bedrock_agent_client.list_agents()
agent_id_list = [
    agent_summary["agentId"] for agent_summary in response["agentSummaries"] if agent_summary["agentName"] == agent_name
]
if agent_id_list:
    logger.info(f"Agent {agent_name} already exists, updating it")
    agent_id = agent_id_list[0]
    response = bedrock_agent_client.update_agent(
        agentId=agent_id_list[0],
        agentName=agent_name,
        agentResourceRoleArn=agent_role["Role"]["Arn"],
        description=agent_description,
        idleSessionTTLInSeconds=1800,
        foundationModel=agent_foundation_model,
        instruction=agent_instruction,
    )
else:
    logger.info(f"Creating agent {agent_name}")
    response = bedrock_agent_client.create_agent(
        agentName=agent_name,
        agentResourceRoleArn=agent_role['Role']['Arn'],
        description=agent_description,
        idleSessionTTLInSeconds=1800,
        foundationModel=agent_foundation_model,
        instruction=agent_instruction,
    )
    logger.info(f"Agent {agent_name} created")
    agent_id = response["agent"]["agentId"]
time.sleep(10)

#### Create Agent Action Group

Now that we have created the agent, let's create an [Action Group](https://docs.aws.amazon.com/bedrock/latest/userguide/agents-action-create.html) and associate with the agent. The action group will allow our agent to get weather. To do so, we will "inform" our agent of existing functionalities using a [function schema](https://docs.aws.amazon.com/bedrock/latest/userguide/agents-action-function.html) defined in `JSON` format.

The function schema requires the function `name`, `description` and `parameters` to be provided. Each parameter has a parameter name, description, type and a boolean flag indicating if the parameter is required.

Let's define the functions `JSON` as `agent_functions`

In [ ]:
agent_functions = [
    {
        'name': 'get_weather',
        'description': 'Give the weather for a city',
        'parameters': {
            "city": {
                "description": "The city to get the weather for",
                "required": True,
                "type": "string"
            }
        }
    },
]

Now we can use the [`create_agent_action_group`](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/bedrock-agent/client/create_agent_action_group.html) function from the boto3 SDK to create the action group

In [ ]:
# Check if the agent action group already exists
agent_version = "DRAFT"
response = bedrock_agent_client.list_agent_action_groups(agentId=agent_id, agentVersion=agent_version)
action_group_id_list = [
    action_group["actionGroupId"]
    for action_group in response["actionGroupSummaries"]
    if action_group["actionGroupName"] == agent_action_group_name
]

# Update or create the agent action group
if action_group_id_list:
    logger.info(f"Agent action group {agent_action_group_name} already exists, updating it")
    agent_action_group_id = action_group_id_list[0]
    agent_action_group_response = bedrock_agent_client.update_agent_action_group(
        agentId=agent_id,
        agentVersion=agent_version,
        actionGroupExecutor={"lambda": lambda_function["FunctionArn"]},
        actionGroupName=agent_action_group_name,
        functionSchema={"functions": agent_functions},
        description=agent_action_group_description,
        actionGroupId=agent_action_group_id
    )
else:
    logger.info(f"Creating agent action group {agent_action_group_name}")
    agent_action_group_response = bedrock_agent_client.create_agent_action_group(
        agentId=agent_id,
        agentVersion=agent_version,
        actionGroupExecutor={"lambda": lambda_function["FunctionArn"]},
        actionGroupName=agent_action_group_name,
        functionSchema={"functions": agent_functions},
        description=agent_action_group_description,
    )

#### Allowing bedrock to invoke lambda function

The last requirement is to add the [resource-based policy](https://docs.aws.amazon.com/bedrock/latest/userguide/agents-permissions.html#agents-permissions-lambda) to allow bedrock to invoke the action group lambda function.

In [ ]:
# Create allow to invoke permission on lambda
lambda_client = boto3.client('lambda', region_name=region)
try:
    response = lambda_client.add_permission(
        FunctionName=lambda_function_name,
        StatementId=f'allow_bedrock_{agent_id}',
        Action='lambda:InvokeFunction',
        Principal='bedrock.amazonaws.com',
        SourceArn=f"arn:aws:bedrock:{region}:{account_id}:agent/{agent_id}",
    )
    print(response)
except lambda_client.exceptions.ResourceConflictException as e:
    print(e)

#### Preparing agent

Before invoking the agent we need to prepare it. Preparing your agent will package all its components, including the security configurations. It will bring the agent into a state where it can be tested in runtime. We will use the [`prepare_agent`](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/bedrock-agent/client/prepare_agent.html) function from the boto3 sdk to prepare our agent.

In [ ]:
response = bedrock_agent_client.prepare_agent(
    agentId=agent_id
)
print(response)
# Pause to make sure agent is prepared
time.sleep(20)

### Invoking Agent

Now that our Agent is ready to be used, let's test it. To do so we will use the [`invoke_agent`](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/bedrock-agent-runtime/client/invoke_agent.html) function from the boto3 Bedrock runtime client.

To invoke an agent, you have to refer to its alias. You can create a new alias, or you can use the test alias to invoke your `DRAFT` agent. The test alias used to invoke the draft agent is `TSTALIASID` and it will work with any agent. 


We will use the support function called `invoke_agent_helper` from the `agents.py` support file to allow us to invoke the agent with or without trace enabled and with or without session state. We will getinto more details about those concepts in the `03_invoke_agent.ipynb` notebook.

In [ ]:
alias_id = "TSTALIASID"

ℹ️ We can use session state (not changing `session_id`) to store information about the conversation and use it in the next invocations. This is useful when you want to keep track of the conversation context.

In [ ]:
session_id:str = str(uuid.uuid1())

In [ ]:
# session_id: str = str(uuid.uuid1())
query = "Quelle est la météo à Paris ?"
response = invoke_agent_helper(query, session_id, agent_id, alias_id, enable_trace=False)
print(response)

In [ ]:
# session_id: str = str(uuid.uuid1())
query = "Quelle est la météo à ?"
response = invoke_agent_helper(
    query, session_id, agent_id, alias_id, enable_trace=True
)

In [ ]:
# session_id: str = str(uuid.uuid1())
query = "Qui est Barack Obama ?"
response = invoke_agent_helper(query, session_id, agent_id, alias_id, enable_trace=False)
print(response)